In [212]:
from loader import DataLoader
from model import Rel2Layout, RelEncoder, Rel2Bbox, Rel2RegLayout
from trainer import Trainer, PretrainTrainer, RegTrainer
from utils import ensure_dir
from loader import RelDataset, Rel2Layout_Dataset
import argparse
import pickle
import json

In [213]:
fn = 'data/vg_anns_45.pkl'
ob_json_path = './data/vg/scene_graph/objects.json'
rel_json_path = './data/vg/scene_graph/relationships.json'
img_json_path = './data/vg/scene_graph/image_data.json'
with open(fn, 'rb') as file:
    all_anns = pickle.load(file)
with open('data/rel_dict_45.pkl', 'rb') as file:
    vocab_dict = pickle.load(file)
with open('data/cls_dict_45.pkl', 'rb') as file:
    cls_dict = pickle.load(file)
with open(ob_json_path) as f:
    ob_json = json.load(f)
with open(rel_json_path) as f:
    rel_json = json.load(f)
with open(img_json_path) as f:
    img_json = json.load(f)

In [224]:
def idx2vocab(idx, modality):
    sent = []
    for i in range(len(idx)):
        if modality == 'text':
            sent.append(vocab_dict[idx[i]])
        else:
            sent.append(cls_dict[idx[i]])
    return sent
print(len(vocab_dict.values()))
print(cls_dict.values())

199
dict_values(['[PAD]', '[CLS]', '[SEP]', '[MASK]', 'window', 'man', 'tree', 'shirt', 'person', 'wall', 'building', 'ground', 'sky', 'sign', 'head', 'grass', 'pole', 'light', 'hand', 'hair', 'leg', 'woman', 'car', 'table', 'trees', 'leaves', 'ear', 'people', 'pants', 'water', 'eye', 'door', 'shadow', 'fence', 'line', 'floor', 'nose', 'chair', 'wheel', 'handle', 'clouds', 'jacket', 'plate', 'arm', 'hat', 'shoe', 'tail', 'face', 'leaf', 'road', 'letter', 'windows', 'sidewalk', 'snow', 'shorts', 'flower', 'bag', 'street', 'helmet', 'field', 'boy', 'cloud', 'glass', 'tile', 'background', 'rock', 'logo', 'stripe', 'tire', 'roof', 'number', 'bottle', 'foot', 'spot', 'post', 'girl', 'mouth', 'umbrella', 'boat', 'part', 'picture', 'legs', 'dirt', 'food', 'glasses', 'cap', 'flowers', 'jeans', 'lights', 'mirror', 'bench', 'top', 'box', 'clock', 'shoes', 'bird', 'bush', 'button', 'brick', 'bowl', 'neck', 'bus', 'edge', 'train', 'trunk', 'shelf', 'lines', 'reflection', 'writing', 'pillow', 'plan

In [225]:
gt_class_id = []
gt_class_name = []

gt_boxes = []
for rel in rel_json[1]['relationships']:
  if rel['subject']['object_id'] not in gt_class_id:
    gt_class_id.append(rel['subject']['object_id'])
    gt_class_name.append(rel['subject']['name'])
    gt_boxes.append([rel['subject']['x'],rel['subject']['y'],rel['subject']['w'],rel['subject']['h']])
  if rel['object']['object_id'] not in gt_class_id:
    gt_class_id.append(rel['object']['object_id'])
    gt_class_name.append(rel['object']['name'])
    gt_boxes.append([rel['object']['x'],rel['object']['y'],rel['object']['w'],rel['object']['h']])
gt_boxes = np.array(gt_boxes)
gt_boxes[:,2:] = gt_boxes[:,:2] + gt_boxes[:,2:]
print(gt_class_id)
print(gt_class_name)
print(gt_boxes)
print(rel_json[1]['relationships'])

[1023838, 5071, 1023836, 1023841, 5074, 5075, 5076, 5077, 1023819, 5080, 5081, 5082, 5083, 5084, 5085, 1023823, 5088, 5089, 5069, 5090]
['man', 'backpack', 'car', 'road', 'tree', 'tree', 'tree', 'crosswalk', 'building', 'window', 'window', 'window', 'window', 'window', 'car', 'sign', 'sneakers', 'bike', 'walk sign', '"walk"']
[[324 320 466 575]
 [359 362 431 443]
 [239 347 375 473]
 [  3 321 375 600]
 [  0 146  73 369]
 [ 58 155 119 356]
 [110 195 193 358]
 [  0 492 366 599]
 [547   4 798 388]
 [645  60 744 239]
 [742  71 799 230]
 [644 259 749 373]
 [746 259 798 367]
 [628  64 654 243]
 [346 327 432 403]
 [230 287 252 354]
 [339 533 471 570]
 [415 403 448 472]
 [463 141 506 244]
 [471 193 502 208]]
[{'predicate': 'wears', 'object': {'name': 'backpack', 'h': 81, 'object_id': 5071, 'synsets': ['backpack.n.01'], 'w': 72, 'y': 362, 'x': 359}, 'relationship_id': 15947, 'synsets': ['wear.v.01'], 'subject': {'name': 'man', 'h': 255, 'object_id': 1023838, 'synsets': ['man.n.01'], 'w': 142, 'y

In [226]:
print(idx2vocab(all_anns['rel'][0],'text'))
print(idx2vocab(all_anns['rel'][0][1::2][all_anns['rel'][0][1::2] != 0],'text'))
print(idx2vocab(all_anns['cls'][0],'image'))
print(all_anns['id'][0])
print(all_anns['image_id'][0])
print(all_anns['image_wh'][0])
print(all_anns['rel_box'][0])
print(len(all_anns['rel']))
print(len(all_anns['image_id']))

['[CLS]', 'car', 'has', 'headlight', '[SEP]', 'sign', 'ON', 'building', '[SEP]', 'man', 'has', 'shirt', '[SEP]', 'sidewalk', 'next to', 'street', '[SEP]', 'car', 'has', 'back', '[SEP]', 'man', 'has', 'glasses', '[SEP]', 'man', 'has', 'shoes', '[SEP]', 'man', 'wears', 'pants', '[SEP]', 'man', 'has', 'jacket', '[SEP]', 'man', 'has', 'pants', '[SEP]', 'bike', 'ON', 'sidewalk', '[SEP]', 'shirt', 'ON', 'man', '[SEP]', 'man', 'WEARING', 'glasses', '[SEP]', 'tree', 'next to', 'street', '[SEP]', 'trees', 'by', 'sidewalk', '[SEP]', 'building', 'with', 'windows', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [227]:
import cv2
from bounding_box import bounding_box as bb
import os

def show_and_save(image, path):
    cv2.imwrite(path, image)

In [228]:
size = [all_anns['image_wh'][0][1],all_anns['image_wh'][0][0],3]
box_mask = np.array(all_anns['rel_box'][0]) != 2.
boxes = np.array(all_anns['rel_box'][0])[box_mask].reshape(-1,4)
id_mask = np.array(all_anns['id'][0]) != 0.
ids = np.array(all_anns['id'][0])[id_mask]
clss = idx2vocab(all_anns['rel'][0][1::2][all_anns['rel'][0][1::2] != 0],'text')

In [229]:
boxes[:,:2] = boxes[:,:2] 
boxes[:,2:] = boxes[:,:2] + boxes[:,2:]

In [230]:
boxes[:,0] *= all_anns['image_wh'][0][0]
boxes[:,1] *= all_anns['image_wh'][0][1]
boxes[:,2] *= all_anns['image_wh'][0][0]
boxes[:,3] *= all_anns['image_wh'][0][1]

In [231]:
print(boxes)
print(gt_boxes)
image = np.full(size, 200.)
gt_image = np.full(size, 200.)

for i in range(len(boxes)):
  bb.add(image, boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3], str(clss[i]))
for i in range(len(gt_boxes)):
  bb.add(gt_image, gt_boxes[i][0], gt_boxes[i][1], gt_boxes[i][2], gt_boxes[i][3], str(gt_class_name[i]))

[[479. 315. 553. 413.]
 [514. 366. 537. 381.]
 [118.  13. 206. 195.]
 [  1.   2. 219. 538.]
 [238. 249. 298. 511.]
 [241. 289. 300. 390.]
 [ 77. 331. 799. 597.]
 [358. 283. 798. 516.]
 [708. 342. 799. 516.]
 [721. 339. 788. 509.]
 [238. 249. 298. 511.]
 [271. 268. 291. 280.]
 [372. 264. 447. 515.]
 [391. 481. 437. 515.]
 [238. 249. 298. 511.]
 [245. 384. 283. 502.]
 [372. 264. 447. 515.]
 [356. 296. 445. 393.]
 [372. 264. 447. 515.]
 [382. 369. 436. 497.]
 [334. 319. 363. 354.]
 [343. 318. 393. 507.]
 [241. 289. 300. 390.]
 [238. 249. 298. 511.]
 [230. 251. 302. 515.]
 [271. 268. 291. 280.]
 [249.   0. 425. 360.]
 [358. 283. 798. 516.]
 [190.   0. 796. 557.]
 [ 77. 331. 799. 597.]
 [  1.   2. 219. 538.]
 [602.   4. 775. 152.]]
[[324 320 466 575]
 [359 362 431 443]
 [239 347 375 473]
 [  3 321 375 600]
 [  0 146  73 369]
 [ 58 155 119 356]
 [110 195 193 358]
 [  0 492 366 599]
 [547   4 798 388]
 [645  60 744 239]
 [742  71 799 230]
 [644 259 749 373]
 [746 259 798 367]
 [628  64 654 24

In [232]:
show_and_save(image, './test.png')
show_and_save(gt_image, './test_gt.png')

In [4]:
import sys
sys.path.append("../")
sys.path.append("./")
from model import Rel2Layout, RelEncoder, Rel2Bbox, Rel2RegLayout
from trainer import Trainer, PretrainTrainer, RegTrainer
from utils import ensure_dir
from loader.COCODataset import COCORelDataset
import argparse
import cv2
from bounding_box import bounding_box as bb
import os
import numpy as np
import pickle
ins_json_path = '../data/coco/instances_train2017.json'
sta_json_path ='../data/coco/stuff_train2017.json' 
vocab_dic_path ='../data/coco/object_pred_idx_to_name.pkl'

COCO = COCORelDataset(instances_json = ins_json_path, stuff_json = sta_json_path, is_mask = False)